# 1.1 Setting up the system for OpenMM (NaCl)

## Imports

In [8]:
## import required packages
import os, math, sys
import simtk.openmm.app  as omm_app
import simtk.openmm as omm
import simtk.unit as unit

### Aims of this Notebook

This is an example notebook on how to set up a simulation system under the OpenMM suite from given Charmm-GUI input topology and parameter files.

You will learn: 
1. Read in topology files (Charmm psf and pdb) for simulation
2. Read in the set of Charmm parameters provided
3. Create and save the OpenMM system for simulation. 
4. Add a custom bias with a harmonic potential 

## Introduction

### Simulating NaCl in water

For the purpose of this workshop, we previously parameterized a Na+ Cl- complex from a PDB crystal structure, solvated the system and generated the necessary files for simulation using Charmm-GUI (topology and parameters which can be found under the **/toppar/** folder). We will be using Charmm36m as our Force Field. If you are interested on generating your own parameters for your system using Charmm-GUI you can find videos/demos and help here: https://www.charmm-gui.org/?doc=demo

We will be sampling the dissociation of NaCl in water and will try to reconstruct the free energy profile from the resulting simulations. In order to sample more efficiently we will use the umbrella sampling method to bias the distance between the ionic complex.  

**Our simulated system looks like this:**

<center>
<img src="images/solvated_system.png" alt="NaCl" width="500"/>
</center>

------

## Hands-on Code

### System creation

We first import the necessary psf/pdb files with the topology info to be able to look up the atom parameters on the force field.

In [9]:
psf_file = 'toppar/step3_input.psf' #Path
pdb_file = 'toppar/step3_input.pdb' #Path

OpenMM has specific functions that read in whatever is necessary from the files:

In [10]:
psf = omm_app.CharmmPsfFile(psf_file)
pdb = omm_app.PDBFile(pdb_file)

From the **/toppar/** folder we read in as well the parameter file with the info for our 2 ions (Na and Cl)

In [11]:
params = omm_app.CharmmParameterSet('toppar/toppar_water_ions.str')

Once everything necessary is in, we can create the system to simulate in OpenMM

In [12]:
## Create an OpenMM system
system = psf.createSystem(params)

### Defining the biasing potential

Now that we have our system we can add our custom bias to it.

In this case since we want to quickly see the dissociation of NaCl, for that, we will be biasing the distance between our two complexed ions. 

The most suitable bias for separating them would be a harmonic biasing potential. All we need to do is tell OpenMM the formula of the biasing potential and the degree of dredom we want to add the biasing potential to, the formula would look like: 

$$
  U_{i} = k_{i}(r - r_{o})^2
$$

In [13]:
## add a harmonic biasing potential on butane dihedral to the OpenMM system

#Define the equation
bias_bond = omm.CustomBondForce("0.5*k*(r-r0)^2")

#Set parameters
bias_bond.addGlobalParameter("k", 1.0)
bias_bond.addGlobalParameter("r0", 0.0) # Initial value of the distance

#Define the bond
bias_bond.addBond(0, 1) #Here you would have to put the atoms you will be adding the bond to

#We add it to the system 
system.addForce(bias_bond)

9

### Saving the system for the next step

Now that all is set up, we can save the file in a readble format. This will be in an XML formated text file, which can be used or re-read at any point later. You can open the file to see how the information is structured for yourself. 

In [14]:
## save the OpenMM system
with open("output_files/NaCl_solvated_system", 'w') as file_handle:
    file_handle.write(omm.XmlSerializer.serialize(system))

-------

## Optional tasks

We recommend if time allows, or at home, to try the following:
